In [1]:
from pymongo import MongoClient
import pandas as pd
client = MongoClient()

db = client.companies

for rest in db.companies.find({"name":"Facebook"}):
    print(rest['name'])

Facebook


- VISUALIZATION PROJECT Geospatial Business Intelligence (BI)
  * Make a geospartial analysis of the `companies` dataset
  * Things you know:
    - You have a software company with 50 employees
    - The company creates video games
    - Roles in your company: 20 developers, 20 Designers/Creatieves/UX/UI and 10 executives/managers
  * Do an analysis about placing the new company offices in the best environment based on the following criteria:
    - There should be software engineers working around
    - The surroundings must have a good ratio of big companies vs startups
    - Ensure you have in your surroundings companies that cover the interests of your team
    - Avoid old companies, prefer recently created ones

In [2]:
molonas = ['web', 'software', 'games_video','mobile', 'music', 'nanotech']

empresas = db.companies.find({'$and': [{'founded_year': {'$gt': 2005 }}, {'number_of_employees': {'$gt': 1}}, {'$or':[{'category_code': 'web' },{'category_code': 'software' },{'category_code': 'games_video' },{'category_code': 'mobile' },{'category_code': 'music' },{'category_code': 'nanotech'}]}]}, {'name':1,'founded_year': 1, '_id': 0, 'number_of_employees':1,'offices.latitude':1, 'offices.longitude':1, 'category_code':1})
                        
#{'number_of_employees': {'$ne': None}}                             
                        



In [3]:
dataset = pd.DataFrame(empresas)
dataset.head(3)

,category_code,founded_year,name,number_of_employees,offices
0,web,2006,Geni,18,"[{'latitude': 34.090368, 'longitude': -118.393..."
1,web,2007,Mahalo,40,"[{'latitude': 34.017606, 'longitude': -118.487..."
2,games_video,2006,Kyte,40,"[{'latitude': 37.788482, 'longitude': -122.409..."


In [4]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2552 entries, 0 to 2551
Data columns (total 5 columns):
category_code          2552 non-null object
founded_year           2552 non-null int64
name                   2552 non-null object
number_of_employees    2552 non-null int64
offices                2552 non-null object
dtypes: int64(2), object(3)
memory usage: 99.8+ KB


In [5]:
dataset['category_code'].unique() 

array(['web', 'games_video', 'mobile', 'software', 'music'], dtype=object)

In [6]:
# Get latitude 
def get_lat(coord): 
    try: 
        return coord[0]['latitude']
    except:
        return None

# Get longitude    
def get_long(coord): 
    try: 
        return coord[0]['longitude']
    except:
        return None

# Inserting new columns with latitude and longitude    
def insert_latlong(df):   
    df['latitude'] = df['offices'].apply(get_lat)
    df['longitude'] = df['offices'].apply(get_long)
    return df


In [7]:
data_limpio = insert_latlong(dataset)
data_limpio.head()

,category_code,founded_year,name,number_of_employees,offices,latitude,longitude
0,web,2006,Geni,18,"[{'latitude': 34.090368, 'longitude': -118.393...",34.090368,-118.393064
1,web,2007,Mahalo,40,"[{'latitude': 34.017606, 'longitude': -118.487...",34.017606,-118.487267
2,games_video,2006,Kyte,40,"[{'latitude': 37.788482, 'longitude': -122.409...",37.788482,-122.409173
3,web,2007,CriticalMetrics,4,"[{'latitude': 37.269175, 'longitude': -119.306...",37.269175,-119.306607
4,games_video,2006,Stickam,35,"[{'latitude': 34.051409, 'longitude': -118.254...",34.051409,-118.254558


In [8]:
data_limpio.drop(['offices'], axis=1)

,category_code,founded_year,name,number_of_employees,latitude,longitude
0,web,2006,Geni,18,34.090368,-118.393064
1,web,2007,Mahalo,40,34.017606,-118.487267
2,games_video,2006,Kyte,40,37.788482,-122.409173
3,web,2007,CriticalMetrics,4,37.269175,-119.306607
4,games_video,2006,Stickam,35,34.051409,-118.254558
5,games_video,2007,Livestream,120,40.726155,-73.995625
6,games_video,2007,Ustream,250,37.392936,-122.079480
7,games_video,2007,AdaptiveBlue,15,40.801358,-74.337200
8,mobile,2006,GrandCentral,3,37.465645,-121.932202
9,web,2007,Pownce,6,37.762541,-122.397224


In [9]:
data_limpio.to_json('data_limpio_ok.json', orient='records')